<a href="https://colab.research.google.com/github/ollihansen90/VectorQuantisierung_Futureskills/blob/main/VecQuant_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kapitel 3: Mathematische Grundlagen
In diesem Notebook sollen die mathematischen Grundlagen für die Vektorquantisierung behandelt werden.

## Setup
Im Setup werden drei Punktewolken generiert und danach eingezeichnet.

In [ ]:
# TODO: Auf dem Jupyter-Hub wird die utils.py lokal gespeichert und muss nicht mit wget von Github gezogen werden.
!wget -nc -q https://raw.githubusercontent.com/ollihansen90/VectorQuantisierung_Futureskills/main/utils.py

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from utils import setup

data, _ = setup()

plt.figure()
plt.scatter(data[:,0], data[:,1])

plt.axis("scaled")
plt.grid();plt.xlabel("x");plt.ylabel("y")
plt.show()

## Der Mittelwert
Für die spätere Berechnung von Clusterzentren werden die Schwerpunkte

Im folgenden Codeblock wird der Mittelwert von Punktewolken "per Hand" berechnet. Hierbei werden zunächst die Punkte alle aufsummiert. Im Anschluss wird diese Summe durch die Anzahl der Punkte geteilt, um den Mittelwert zu erhalten.

Beim Generieren des Clusters oben rechts wurden Punkte aus einer Normalverteilung gezogen, die den Mittelwert `[5,5]` hat. Der sich ergebene Mittelwert sollte also nah an diesem Vektor liegen. Warum ist der Mittelwert nicht *genau* auf `[5,5]`? Es handelt sich bei dem Mittelwert, wie er hier berechnet wird, lediglich um eine *Schätzung*. Wären für das Cluster unendlich viele Punkte gegeben, so wäre auch der berchnete Mittelwert `[5,5]`.

In [ ]:
# Mittelwert Blob 1
summe = np.zeros(2)
blob1 = data[:int(len(data)/3)]

for point in blob1:
    summe = summe+point
mittelwert = summe/len(blob1)

print(mittelwert)

plt.figure()
plt.scatter(data[:,0], data[:,1])
plt.scatter(*mittelwert)
plt.axis("scaled")
plt.grid();plt.xlabel("x");plt.ylabel("y")
plt.show()

## Exponentielles Mittel
Es gibt Umstände, unter denen zum Zeitpunkt der Berechnung des Mittelwertes nicht alle Datenpunkte vorliegen. Das klassische Beispiel hier wäre ein Datensatz als Zeitreihe, wie beispielsweise das Wetter. Soll der Mittelwert der Temperatur berechnet werden, so kann nicht heute schon auf die Temperatur von morgen zugegriffen werden.

Für diesen Fall gibt es das *exponentielle Mittel* (engl. Exponential Moving Average, kurz EMA). Die Idee beim exponentiellen Mittel ist, dass der bisherige Mittelwert nur leicht in Richtung des aktuellen Punktes verschoben wird. Beim Wetterbeispiel könnte das so aussehen, dass wir für diese Woche bereits einen EMA von 20°C bestimmt haben. Die heutige Temperatur beträgt 24°C, wir aktualisieren den EMA also leicht nach oben auf 20.4°C.

Mathematisch wird diese Formel rekursiv aufgestellt:

$$x_{neu}=x_{alt}+0.1\cdot(x_{heute}-x_{alt}) = 20+0.1\cdot(24-20)=20.4$$

Bei $x_{neu}$ und $x_{alt}$ handelt es sich um die Temperaturen laut exponentiellem Mittel, $x_{heute}$ ist die heutige Temperatur. $x_{heute}-x_{alt}$ ist die Temperaturdifferenz, über die das neue Mittel angepasst wird.

### Aufgabe
Im folgenden Codeblock wird der Mittelwert iterativ über das exponentielle Mittel angepasst. Hierbei läuft die Anpassung über zwei $Epochen$. Eine Epoche ist so definiert, dass jeder Punkt in dem Datensatz (hier nur das Cluster oben rechts) ein Mal für einen Updateschritt genutzt wird.

Wählen Sie unterschiedliche Startwerte für das exponentielle Mittel und beobachten Sie, wie das exponentielle mittel gegen das Cluster oben rechts konvergiert.

In [ ]:
# Mittelwert exponentiell Blob 1
n_epochs = 2
startwert = np.array([-10,10])

mittelwertliste = np.zeros([n_epochs*len(blob1),2])
mittelwert = startwert
lr = 0.1
t = 0
for epoch in range(n_epochs):
    for i, point in enumerate(blob1[np.random.permutation(len(blob1))]):
        mittelwert = mittelwert+lr*(point-mittelwert)
        mittelwertliste[t] = mittelwert
        t += 1

print(mittelwert)
plt.figure()
plt.scatter(data[:,0], data[:,1])
plt.plot(mittelwertliste[:,0], mittelwertliste[:,1], "tab:orange")
plt.scatter(mittelwert[0], mittelwert[1])
plt.axis("scaled")
plt.grid();plt.xlabel("x");plt.ylabel("y")
plt.show()